**Environment Dependencies**

In [ ]:
!pip install numpy==1.17.5;
import os
os.kill(os.getpid(), 9)

In [1]:
%tensorflow_version 1.x
!pip install tf_slim

TensorFlow 1.x selected.
     |████████████████████████████████| 352 kB 5.2 MB/s 


In [1]:
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
# A step means using a single batch of data. larger batch, less steps required
#60000 steps is required to train our example sign language dataset, less or more may be required based on your custom datasets needs

num_steps = 5000  


#Number of evaluation steps.
num_eval_steps = 50

#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    }
}
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

**Object detection API**

In [2]:
# %%capture
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

Traceback (most recent call last):
  File "object_detection/builders/model_builder_test.py", line 21, in <module>
    from object_detection.builders import model_builder
  File "/content/models/research/object_detection/builders/model_builder.py", line 19, in <module>
    from object_detection.builders import anchor_generator_builder
  File "/content/models/research/object_detection/builders/anchor_generator_builder.py", line 23, in <module>
    from object_detection.anchor_generators import flexible_grid_anchor_generator
  File "/content/models/research/object_detection/anchor_generators/flexible_grid_anchor_generator.py", line 19, in <module>
    from object_detection.anchor_generators import grid_anchor_generator
  File "/content/models/research/object_detection/anchor_generators/grid_anchor_generator.py", line 27, in <module>
    from object_detection.utils import ops
  File "/content/models/research/object_detection/utils/ops.py", line 28, in <module>
    import tf_slim as slim
Mo

In [3]:
import os

repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull


/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.83 MiB | 38.41 MiB/s, done.
Resolving deltas: 100% (428/428), done.
/content/tensorflow-object-detection-faster-rcnn
Already up to date.


Prepare tfRecord

In [4]:
%cd /content/tensorflow-object-detection-faster-rcnn/data

!curl -L "[https://universe.roboflow.com/ds/dAYeXVi9Zr?key=Am6Gb6n0B6]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content/tensorflow-object-detection-faster-rcnn/data
curl: (3) [globbing] bad range in column 2
Archive:  roboflow.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of roboflow.zip or
        roboflow.zip.zip, and cannot find roboflow.zip.ZIP, period.


In [5]:
#RENAME Based on your annotation type - In this example our annotation type is "letters"
#RENAME to the file path based on your download above ^^
test_record_fname = '/content/tensorflow-object-detection-faster-rcnn/data/test/Letters.tfrecord'
train_record_fname = '/content/tensorflow-object-detection-faster-rcnn/data/train/Letters.tfrecord'
label_map_pbtxt_fname = '/content/tensorflow-object-detection-faster-rcnn/data/train/Letters_label_map.pbtxt'